

## <font color='blue'>Lab 2</font>
## <font color='blue'>Construindo Seu Próprio ChatGPT Para Geração de Texto com PyTorch</font>

In [1]:
!pip install -q transformers==4.28.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.2 MB/s eta 0:00:00


In [ ]:
!pip install -q torch==2.0.0

In [ ]:
%env TF_CPP_MIN_LOG_LEVEL=3

env: TF_CPP_MIN_LOG_LEVEL=3


In [ ]:
# Importação das bibliotecas necessárias
import torch
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

transformers: 4.28.1
torch       : 2.0.0



In [ ]:
# Função para geração de texto
def gera_texto(prompt, model, tokenizer, device, max_length = 50, num_return_sequences = 1):

    # Coloca o modelo em modo de previsão
    model.eval()

    # Extrai os tokens do texto de entrada no formato do PyTorch (pt)
    input_ids = tokenizer.encode(prompt, return_tensors = 'pt').to(device)

    # Gera sequências de texto a partir do texto inicial
    generated_sequences = model.generate(input_ids = input_ids,
                                         max_length = max_length,
                                         num_return_sequences = num_return_sequences,
                                         no_repeat_ngram_size = 2,
                                         pad_token_id = tokenizer.eos_token_id,
                                         do_sample = True,
                                         top_k = 50,
                                         top_p = 0.95,
                                         temperature = 0.8)

    return [tokenizer.decode(sequence, skip_special_tokens = True) for sequence in generated_sequences]

No código acima, model.generate() é uma função usada para gerar sequências de texto a partir de um modelo de linguagem. Cada argumento que a função recebe tem um propósito específico:

**input_ids**: Este é o identificador de entrada para o modelo. Geralmente é uma sequência de números que representam o texto de entrada, transformado em tokens por um tokenizer.

**max_length**: Este é o comprimento máximo da sequência de saída. O modelo irá parar de gerar quando atingir este limite.

**num_return_sequences**: Este é o número de sequências independentes a serem retornadas. Cada uma será gerada de um único prompt.

**no_repeat_ngram_size**: Esse é o tamanho do n-grama que não deve ser repetido. Se for 2, por exemplo, o modelo não gerará a mesma sequência de 2 tokens seguidos.

**pad_token_id**: Este é o ID do token que será usado para preenchimento (padding) se a sequência gerada for mais curta que o max_length.

**do_sample**: Se definido como True, a função irá usar amostragem aleatória para gerar o texto, o que significa que poderá haver variações a cada vez que a mesma entrada for fornecida. Se False, a função sempre gerará a mesma saída para uma determinada entrada.

**top_k**: Este parâmetro é usado para limitar a amostragem às K principais probabilidades. O modelo seleciona um dos K tokens mais prováveis para continuar a sequência a cada passo.

**top_p**: Este é o valor de "nucleus sampling" ou "top-p sampling". Ao invés de selecionar as top K probabilidades, o modelo seleciona um conjunto mínimo de tokens que tem uma probabilidade cumulativa de pelo menos p. Isso pode levar a um conjunto de tokens potencialmente menor ou maior do que K, dependendo da distribuição de probabilidade.

**temperature**: Este parâmetro afeta a "aleatoriedade" das escolhas do modelo. Valores mais altos fazem a distribuição de probabilidade mais plana (mais aleatória), enquanto valores mais baixos tornam a distribuição mais aguda (menos aleatória). Por exemplo, uma temperatura de 0.1 fará o modelo quase sempre escolher o token mais provável, enquanto uma temperatura de 1.0 fará a seleção ser mais proporcional às probabilidades dos tokens.

https://huggingface.co/gpt2

In [ ]:
# Modelo
PRETRAINED_MODEL = 'gpt2'

In [ ]:
# Dispositivo
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Inicialização do tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(PRETRAINED_MODEL)

In [ ]:
# Inicialização do modelo
modelo = GPT2LMHeadModel.from_pretrained(PRETRAINED_MODEL)

In [ ]:
# Coloca o modelo no device
modelo.to(DEVICE)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# Texto inicial
prompt = 'The advantages of artificial intelligence are'

In [ ]:
# Gerando texto
generated_texts = gera_texto(prompt,
                             modelo,
                             tokenizer,
                             DEVICE,
                             max_length = 100,
                             num_return_sequences = 3)

In [ ]:
# Imprime o resultado
for idx, generated_text in enumerate(generated_texts):
    print(f'Texto Gerado {idx + 1}:\n{generated_text}\n')

Texto Gerado 1:
The advantages of artificial intelligence are myriad, and Amendola is likely to be the first to admit that the future is bright.

Texto Gerado 2:
The advantages of artificial intelligence are that it is not constrained by a single set of laws and that there is no need to think in isolation. It can be applied in many different ways, and some of the most useful of them could be implemented in the real world. In fact, the human mind is so much more complex than that of a computer. There are many computational tools that can do a lot of very important things, from designing algorithms to developing artificial neural networks, but we are not fully familiar

Texto Gerado 3:
The advantages of artificial intelligence are that it is already able to understand human behavior and use information on it to do its job, whereas most humans are limited to doing their jobs in front of computers. However, human intelligence does need to have some kind of capability for human interaction,

# Fim